In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import re
from collections import defaultdict
from copy import deepcopy
import torch

torch.cuda.is_available()
import yaml
import pyhocon
import joblib
from copy import deepcopy
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from RobotCommandClassifier import utils
from RobotCommandClassifier import *
from MultilabelML import PrepareData, PrepareInput_for_simpletransformers, PrepareModel

In [175]:
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/SimpleLM.conf")
#CONFIG = configFileContent['rubert_tiny2_multilabel_noYno0_fold4'].as_plain_ordered_dict()
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
#CONFIG = configFileContent['MyMultiTiny2_data2_fold0'].as_plain_ordered_dict()
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiMLangBert_data3v2/fold_2/config.json", "r") as f:
    CONFIG = json.load(f)

# если хотим протестировать на всем тесте, включая фолды, на которых он обучался
#CONFIG["Data"].pop("test_only_on_fold")

In [176]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = PrepareData(CONFIG)

In [177]:
sorted([x for x in os.listdir(CONFIG["output_dir"] + '/models/') if "epoch" in x])

['checkpoint-10964-epoch-2',
 'checkpoint-16446-epoch-3',
 'checkpoint-21928-epoch-4',
 'checkpoint-5482-epoch-1']

In [178]:
epoch_i = 10
# MyMultiTiny2
epoch_to_checkpoint = {1:2058,2:4116,3:6174,4:8232,5:10290,6:12348,7:14406,8:16464,9:18522,10:20580}
# MyMultiTiny2_data2
epoch_to_checkpoint = {1:6378,2:12756,3:19134,4:25512,5:31890,6:38268,7:44646,8:51024,9:57402,10:63780}
#MyMultiTiny2_data3
epoch_to_checkpoint = {1:5481,2:10962,3:16443,4:21924,5:27405,6:32886,7:38367,8:43848,9:49329,10:54810,
                      11:60291, 12:65772, 13:71253, 14:76734, 15:82215, 16:87696, 17:93177, 18:98658, 19:104139, 20:109620}
#MyMultiTiny2_data3 fold 3, 4
epoch_to_checkpoint = {1:5482,2:10964,3:16446,4:21928,5:27410,6:32892,7:38374,8:43856,9:49338,10:54820,
                      11:60302, 12:65784, 13:71266, 14:76748, 15:82230, 16:87712, 17:93194, 18:98676, 19:104158, 20:109640}
CONFIG["Model"]['model_name'] = CONFIG["output_dir"] + '/models/checkpoint-{}-epoch-{}'.format(epoch_to_checkpoint[epoch_i], epoch_i)

model = PrepareModel(CONFIG)

OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like /s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiMLangBert_data3/fold_4/models/checkpoint-54820-epoch-10 is not the path to a directory containing a {configuration_file} file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

# Make prediction

In [ ]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())
#predictions = np.array(predictions)

In [ ]:
#привести бинарный мультилейбл к мультиклассовому
classes_per_attribute = CONFIG["Model"]['num_sublabels_per_biglabel']
predictions_2 = utils.binarymultilabel_to_multiclassmultilabel(raw_outputs, classes_per_attribute, CONFIG)

In [ ]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_classes_report.json".format(epoch_i)), "w") as f:
    json.dump(result, f)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_avg_report.json".format(epoch_i)), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

In [ ]:
possible_combinations = pd.read_csv("../Data/Interim/possible_combinations_of_attributes_for_actions.csv")
possible_combinations_arr = possible_combinations.loc[:, CONFIG["Data"]["target_columns"]].values

In [ ]:
# Rule
def softmax(x):
    #e_x = x - np.expand_dims(np.max(x, axis=1), axis=1)
    #return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)
    e_x = np.exp(x)
    return e_x / np.expand_dims(np.sum(e_x, axis=1), axis=1)
softmax_outputs = np.zeros_like(raw_outputs, dtype=np.float32)
shift = 0
for num_sublabels in CONFIG['Model']['num_sublabels_per_biglabel']:
    softmax_outputs[:, shift:shift + num_sublabels] = softmax(raw_outputs[:, shift:shift + num_sublabels])
    shift += num_sublabels

predictions_3 = []
for i in range(len(softmax_outputs)):
    shift = 0
    probs_for_combinations = np.zeros_like(possible_combinations_arr, dtype=np.float32)
    # предполагается, что первые значения в выходном векторе из модели - классы экшена
    assert CONFIG["Data"]["target_columns"][0]=="action"
    probs_for_combinations[:,0] = np.take(softmax_outputs[i, shift:shift+classes_per_attribute[0]], possible_combinations_arr[:,0])
    shift += classes_per_attribute[0]
    #print(probs_for_combinations[:,0])
    maxprob_attribute_classes = [-1] # -1 for action
    for attribute_i in range(1, len(classes_per_attribute)):
        # есть вариант ставить 0 для нулевых классов или наоборот - обратное от максимального класса
        probs = softmax_outputs[i, shift:shift+classes_per_attribute[attribute_i]]
        assert np.round(sum(probs), 5)==1
        zerocls_prob = probs[0]
        nonzerocls_max_prob = np.max(probs[1:])
        #print(zerocls_prob, nonzerocls_max_prob)
        maxprob_attribute_classes.append(np.argmax(probs[1:])+1)
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==0, attribute_i] = zerocls_prob
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==1, attribute_i] = nonzerocls_max_prob
        shift += classes_per_attribute[attribute_i]

    best_template_i = np.argmax(np.sum(probs_for_combinations, axis=1))
    sample_prediction = possible_combinations_arr[best_template_i].copy()
    for i in range(1, len(classes_per_attribute)):
        if sample_prediction[i]!=0:
            sample_prediction[i]=maxprob_attribute_classes[i]
    predictions_3.append(sample_prediction)
predictions_3 = np.array(predictions_3)

In [ ]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
result = utils.calculate_metrics_2(test_y_df, predictions_3, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_classes_report_rule.json".format(epoch_i)), "w") as f:
    json.dump(result, f)

result_avg = utils.calculate_metrics(test_y_df, predictions_3, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-{}_avg_report_rule.json".format(epoch_i)), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)